In [9]:
import pickle
import string
import annoy

import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from pymorphy2 import MorphAnalyzer
from stop_words import get_stop_words
from gensim.models import FastText
from tqdm import tqdm_notebook

In [11]:
checks = pd.read_csv("/content/drive/MyDrive/чековые данные.csv")
checks.drop(['name'], axis=1, inplace=True)

with open("/content/drive/MyDrive/Product_dict.pkl","rb") as f:
    product_dict = pickle.load(f)

checks.head()

<ipython-input-11-ca4abf38e125>:1: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  checks = pd.read_csv("/content/drive/MyDrive/чековые данные.csv").head(100)


,sale_date_date,contact_id,shop_id,product_id,product_sub_category_id,product_category_id,brand_id,quantity
0,2018-12-07,1260627,1455.0,168308.0,906.0,205.0,-1.0,"1,00"
1,2018-12-07,198287,279.0,134832.0,404.0,93.0,-1.0,"1,00"
2,2018-12-07,2418385,848.0,101384.0,404.0,93.0,-1.0,"1,00"
3,2018-12-07,1285774,1511.0,168570.0,906.0,205.0,-1.0,"1,00"
4,2018-12-07,1810323,1501.0,168319.0,906.0,205.0,-1.0,"1,00"


In [12]:
# Создадим ключевую колонку: время + касса + магазин
checks['key_col'] = checks['sale_date_date'].apply(str) + "_" + checks['contact_id'].apply(str) + "_" + checks['shop_id'].apply(str)
checks.head()

,sale_date_date,contact_id,shop_id,product_id,product_sub_category_id,product_category_id,brand_id,quantity,key_col
0,2018-12-07,1260627,1455.0,168308.0,906.0,205.0,-1.0,"1,00",2018-12-07_1260627_1455.0
1,2018-12-07,198287,279.0,134832.0,404.0,93.0,-1.0,"1,00",2018-12-07_198287_279.0
2,2018-12-07,2418385,848.0,101384.0,404.0,93.0,-1.0,"1,00",2018-12-07_2418385_848.0
3,2018-12-07,1285774,1511.0,168570.0,906.0,205.0,-1.0,"1,00",2018-12-07_1285774_1511.0
4,2018-12-07,1810323,1501.0,168319.0,906.0,205.0,-1.0,"1,00",2018-12-07_1810323_1501.0


In [13]:
checks.dropna(inplace=True)
checks = checks[checks['sale_date_date'].str.startswith("20")]
checks.reset_index(inplace=True, drop=True)
checks['sale_date_date'] = pd.to_datetime(checks['sale_date_date'])
checks.sort_values('sale_date_date', inplace=True)

# Достаточно отсортировать и сделать трейн-тест, чтобы построить валидацию. Забьем на разорванные сессии
train, test = train_test_split(checks, test_size=0.3)

In [17]:
# Обучим простые контетные рекомендации

# Для фильтрации пунктуации
exclude = set(string.punctuation)
# Для приведения слов в начальной форме
morpher = MorphAnalyzer()

# Для фильтрации стоп-слов
sw = get_stop_words("ru")

def preprocess_txt(line):
    spls = "".join(i for i in str(line).strip() if i not in exclude).split()
    spls = [morpher.parse(i.lower())[0].normal_form for i in spls]
    spls = [i for i in spls if i not in sw and i != ""]
    return spls

sentences = [preprocess_txt(k) for k in product_dict.keys()]

# Подготовим кандидатогенератор, который будет отдавать фильмы похожие по текстовому описанию на те, .
# которые оенил пользователь
# Обучим Fasttext и заэмбедим фильмы
modelFT = FastText(sentences=sentences, min_count=1, window=5)

# Для того, чтобы быстро находить айтемы положим эмбединги их тайтлов в ANN индекс
# Создадим объект индекса
ft_index_names = annoy.AnnoyIndex(20 ,'angular')

# Будем хранить соответствия не только id-> фильм, но и фильм-> id, чтобы быстрее находить эмбеддинги айтемов
reverse_index_map = {}
counter = 0

for name in tqdm_notebook(product_dict.keys()):
    n_ft = 0
    reverse_index_map[counter] = name
    vector_ft = np.zeros(20)
    # Каждое слово обернем в эмбеддинг
    for word in preprocess_txt(name):
        n_ft += 1
    if n_ft > 0:
        vector_ft = vector_ft / n_ft
    ft_index_names.add_item(counter, vector_ft)
    counter += 1


<ipython-input-17-7aca30a6f88d>:32: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for name in tqdm_notebook(product_dict.keys()):


  0%|          | 0/30418 [00:00<?, ?it/s]

In [18]:
reverse_item_mapping = {v: k for k, v in product_dict.items()}

def recommend(items_list):
    current_vector = np.zeros(20)
    c = len(items_list)
    for iid in items_list:
        iname = product_dict[iid]
        if iname not in reverse_index_map:
            c -= 1
            continue
        current_vector += np.array(ft_index_names.get_item_vector[reverse_index_map[iname]])
    return [product_dict[reverse_index_map[i]] for i in ft_index_names.get_nns_by_vector(current_vector / c, 10)]

In [35]:
recommend(['52539', '110342'])

<ipython-input-18-e72575b33fe8>:12: RuntimeWarning: invalid value encountered in true_divide
  return [product_dict[reverse_index_map[i]] for i in ft_index_names.get_nns_by_vector(current_vector / c, 10)]


[]